In [78]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm.notebook import tqdm

import os

import pickle


processed = pd.read_csv("../Dataset/processedDataWithThaliaAdded.csv", sep="|")
book = pd.read_csv("../Dataset/book.csv", sep="|")

In [2]:
## Load pickle
with open("../Dataset/From Other Teams/team404.pickle","rb") as fr:
    data404 = pickle.load(fr)

(204272, 49)


,itemID,isbn,title,authors,publisher,main topic,subtopics,google_title,google_subtitle,google_authors,...,thalia_average_rating,thalia_total_rating,thalia_publisher,thalia_sales_rank,thalia_coverlink,thalia_also_read,thalia_bought_with,thalia_genres,thalia_min_age,thalia_max_age
0,21310,9780552555968,Princess Poppy: The Big Mix Up,[Janey Louise Jones],Penguin Random House Children's UK,YFB,[5AH],The Big Mix-up,<NA>,[Janey Louise Jones],...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1536,34217,9783641195908,Ewig geliebt,[J. R. Ward],Heyne Taschenbuch,FMR,"[1KBB-US-NAK, FMX, FRX, 3MRBF]",Ewig geliebt,Black Dagger 28 - Roman,[J. R. Ward],...,4.8,9,Random House ebook,13994,https://assets.thalia.media/img/artikel/f3f802...,"[9783641206246, 9783732008094, 9783736303522, ...",9783641170226,"[Erotik, Romane & Erzählungen, Fantasy & Scien...",<NA>,<NA>
1537,31436,9783845817446,Meine Sticker-Tiere,<NA>,Ars Edition GmbH,YBG,"[5AD, YBG, YBLL]",Meine Sticker-Tiere,Mit über 400 Stickern,<NA>,...,5.0,1,arsedition,38013,https://assets.thalia.media/img/artikel/c23fa2...,"[9783845812137, 9781782326816, 9783845833675, ...",9783845809670,<NA>,4,6


In [80]:
book.head(3)

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,5AH
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,5AJ AGZ WFA YBG YBL YNA YPA ...
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,5AP FBA


In [81]:
processed.head(3)

,itemID,title,titleFound,title_thalia,author_raw,author_api,author_thalia,language,language_thalia,collection,...,publisher_raw,publisher_api,publishedDate,description,main topic,subtopics,thumbnail,maturityRating,pageCount,isEbook
0,5649,... and the Word became a Story,... and the Word became a Story,and the word became a story,The Author,The Author,the author,en,Englisch,2.0,...,Books on Demand,BoD – Books on Demand,2018-04-17,At the beginning was the Word. And from the Wo...,FL,FL FM FN,http://books.google.com/books/content?id=FrElD...,NOT_MATURE,304.0,False
1,77497,... EVVAI!,... EVVAI!,evvai,Cristina Polacchini,CRISTINA POLACCHINI,cristina polacchini,it,Italienisch,2.0,...,Lulu.com,Lulu.com,2015-08-01,"Amicizia, scuola, amore, alti e bassi, in and ...",YFB,,http://books.google.com/books/content?id=lGNgC...,NOT_MATURE,106.0,False
2,13238,'... then he ate my boy entrancers.',Then He Ate My Boy Entrancers,then he ate my boy entrancers,Louise Rennison,NaN,louise rennison,un,Englisch,2.0,...,HarperCollins Publishers,NaN,2008,NaN,YFM,4Z GB ACN YFQ,NaN,NOT_MATURE,NaN,False


In [82]:
book['key'] = ((book['title'] + ' '+ book['author']).astype(str)).str.lower()
book['key'] = book['key'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

processed['key'] = ((processed['title'] + ' '+ processed['author_raw']).astype(str)).str.lower()
processed['key'] = processed['key'].str.replace(pat=r'[^\w]', repl=r'', regex=True)

In [83]:
remained = book[~book['key'].isin(processed['key'].tolist())]
remained.reset_index(inplace=True)
remained.drop(['index'], axis=1, inplace=True)
remained

C:\Users\rmaql\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,itemID,title,author,publisher,main topic,subtopics,key
0,14576,Unsterblich 01 - Tor der Dämmerung,Julie Kagawa,Heyne Taschenbuch,YFE,5AQ FM YFE YFH,unsterblich01torderdämmerungjuliekagawa
1,15581,Der Herr der Ringe,J. R. R. Tolkien,Klett-Cotta Verlag,FMB,FYT YFH,derherrderringejrrtolkien
2,64544,Spiele-Klassiker Gummitwist-Set,Birgit Fuchs,moses. Verlag GmbH,YBL,,spieleklassikergummitwistsetbirgitfuchs
3,23197,Nichts,Janne Teller,dtv Verlagsgesellschaft,YFS,5AQ 1DND YFB YFS,nichtsjanneteller
4,2417,A Court of Thorns and Roses. Acotar Adult Edition,Sarah J. Maas,Bloomsbury UK,YFH,FMR,acourtofthornsandrosesacotaradulteditionsarahj...
...,...,...,...,...,...,...,...
16110,17607,"Ysstrhm 4, Second Quest",Douglas Browning,Xlibris,FM,,ysstrhm4secondquestdouglasbrowning
16111,70985,"Ysstrhm 5, Flight's End",Douglas Browning,Xlibris,FM,,ysstrhm5flightsenddouglasbrowning
16112,43737,The Box and the Shiny Red Ball Part 3,Mark A. Fettig,Xlibris,FL,,theboxandtheshinyredballpart3markafettig
16113,37678,Timeless Fairy Tales,"Brothers Grimm, Marie-Catherine Baroness D'Aulnoy",MEDIAMORPHOSIS,YFA,YFJ,timelessfairytalesbrothersgrimmmariecatherineb...


In [84]:
mask = data404['itemID'].isin(remained['itemID'])
booksInCommon, left = data404[mask], data404[~mask]

In [85]:
booksInCommon

,itemID,isbn,title,authors,publisher,main topic,subtopics,google_title,google_subtitle,google_authors,...,thalia_average_rating,thalia_total_rating,thalia_publisher,thalia_sales_rank,thalia_coverlink,thalia_also_read,thalia_bought_with,thalia_genres,thalia_min_age,thalia_max_age
1538,14576,9783641100148,Unsterblich 01 - Tor der Dämmerung,[Julie Kagawa],Heyne Taschenbuch,YFE,"[5AQ, FM, YFE, YFH]",Unsterblich - Tor der Dämmerung,Band 1 - Roman,[Julie Kagawa],...,4.5,43,Random House ebook,15835,https://assets.thalia.media/img/artikel/b178ab...,"[9783641171810, 9783736306103, 9783732500239, ...",9783641122713,"[Jugendbücher, Romane & Erzählungen, Fantasy &...",14,<NA>
1553,15581,9783608105353,Der Herr der Ringe,[J. R. R. Tolkien],Klett-Cotta Verlag,FMB,"[FYT, YFH]",Der Herr der Ringe,In der Übersetzung von Margaret Carroux,[J.R.R. Tolkien],...,4.7,62,Klett Cotta,6415,https://assets.thalia.media/img/artikel/373ed8...,"[9783608101393, 9783426451311, 9783641139582, ...",9783608101386,"[Fantasy & Science Fiction, Fantasy, Kinderbüc...",<NA>,<NA>
1576,4586,9783966982573,Meine Freunde,[Viktoria Schlünzen],NOVA MD,YZG,"[5AF, YZS]",Meine Freunde,Ein Freundschaftsbuch - Freundebuch für Kinder...,[Viktoria Schlünzen],...,5.0,1,Nova Md,22875,https://assets.thalia.media/img/artikel/13d2f4...,"[4014489123606, 9783314104534, 4142777520001, ...",9783946739951,"[Kinderbücher, Reise & Abenteuer, Ratgeber, Fa...",5,7
1586,5368,9781444942552,Winter Stories,[Enid Blyton],Hachette Children's Book,YFA,"[5AH, 5HPD, 4CD, YFH, YFJ, YFU]",Winter Stories,Contains 25 Classic Tales,[Enid Blyton],...,NaN,<NA>,Hachette Children's Group,<NA>,https://assets.thalia.media/img/artikel/aad0ee...,"[9780140309577, 9780679883388, 9780007274772, ...",9783125737686,<NA>,6,11
1620,73840,9780711252172,Astrid Lindgren,[María Isabel Sánchez Vegara],Insel Verlag GmbH,JBSF1,"[5AF, JBSF1, YBCS, YBL, YNB, YNG, YNL, YNM, YN...",Astrid Lindgren,<NA>,[Maria Isabel Sanchez Vegara],...,NaN,<NA>,Frances Lincoln,<NA>,https://assets.thalia.media/img/artikel/269656...,"[9780711245730, 9780711243170, 9780711257580, ...",9783836582506,<NA>,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66052,14187,9781450008341,Remnant,[Lesley Barklay],Xlibris,FM,<NA>,REMNANT,<NA>,[Lesley Barklay],...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66053,17607,9781450008501,"Ysstrhm 4, Second Quest",[Douglas Browning],Xlibris,FM,<NA>,"Ysstrhm 4, Second Quest",<NA>,[Douglas Browning],...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66054,70985,9781450012119,"Ysstrhm 5, Flight's End",[Douglas Browning],Xlibris,FM,<NA>,"Ysstrhm 5, Flight's End",<NA>,[Douglas Browning],...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66059,37678,9789738882690,Timeless Fairy Tales,"[Brothers Grimm, Marie-Catherine Baroness D'Au...",MEDIAMORPHOSIS,YFA,[YFJ],Timeless Fairy Tales,<NA>,"[Jacob Grimm, Wilhelm Grimm, Marie-Catherine B...",...,NaN,<NA>,Mediamorphosis,<NA>,https://assets.thalia.media/img/artikel/29a064...,"[9780141331201, 9780982499474, 9781680100280, ...",<NA>,<NA>,<NA>,<NA>


In [86]:
leftout = remained[~remained['itemID'].isin(booksInCommon['itemID'].tolist())]
leftout.reset_index(inplace=True)
leftout.drop(['index'], axis=1, inplace=True)
leftout

C:\Users\rmaql\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,itemID,title,author,publisher,main topic,subtopics,key
0,64544,Spiele-Klassiker Gummitwist-Set,Birgit Fuchs,moses. Verlag GmbH,YBL,,spieleklassikergummitwistsetbirgitfuchs
1,23197,Nichts,Janne Teller,dtv Verlagsgesellschaft,YFS,5AQ 1DND YFB YFS,nichtsjanneteller
2,2417,A Court of Thorns and Roses. Acotar Adult Edition,Sarah J. Maas,Bloomsbury UK,YFH,FMR,acourtofthornsandrosesacotaradulteditionsarahj...
3,36495,Gregs Tagebuch - Band 1 und 2,Jeff Kinney,Baumhaus Verlag GmbH,YFS,5AL 5AM 1KBB YFS 3MR,gregstagebuchband1und2jeffkinney
4,19898,Jupiter,Brandon Q. Morris,Belle Epoque Verlag,FLP,5AT 1ZMCK FLU,jupiterbrandonqmorris
...,...,...,...,...,...,...,...
4965,24751,Die Gier nach Reichtum,Hans Lebek,AAVAA Verlag UG,FL,,diegiernachreichtumhanslebek
4966,38138,Imperium,John P. Johnston,"Kessinger Publishing, LLC",FUP,,imperiumjohnpjohnston
4967,46640,Modern Philistines,J. Benson Hamilton,"Kessinger Publishing, LLC",FRT,,modernphilistinesjbensonhamilton
4968,72005,Denver D.C.,J. a. Burgess,Xlibris,FL,,denverdcjaburgess


In [60]:
forQuery = booksInCommon.copy()
# forQuery.columns[7:32]
forQuery = forQuery.drop(['google_title', 'google_subtitle', 'google_authors', 'google_publisher',
       'google_publishedDate', 'google_description', 'google_logPageCount',
       'google_genre', 'google_averageRating', 'google_ratingsCount',
       'google_isMature', 'google_language', 'goodreads_title',
       'goodreads_authors', 'goodreads_series', 'goodreads_successor',
       'goodreads_description', 'goodreads_language', 'goodreads_setting',
       'goodreads_top_genre', 'goodreads_other_genres',
       'goodreads_average_rating', 'goodreads_total_rating',
       'goodreads_also_read', 'goodreads_coverlink'], axis = 1)
forQuery.columns

Index(['itemID', 'isbn', 'title', 'authors', 'publisher', 'main topic',
       'subtopics', 'thalia_title', 'thalia_subtitle', 'thalia_authors',
       'thalia_serie', 'thalia_successor', 'thalia_description',
       'thalia_languages', 'thalia_average_rating', 'thalia_total_rating',
       'thalia_publisher', 'thalia_sales_rank', 'thalia_coverlink',
       'thalia_also_read', 'thalia_bought_with', 'thalia_genres',
       'thalia_min_age', 'thalia_max_age'],
      dtype='object')

In [63]:
forQuery = booksInCommon.copy()
forQuery = forQuery.drop(forQuery.columns[7:32], axis = 1)
forQuery.columns

Index(['itemID', 'isbn', 'title', 'authors', 'publisher', 'main topic',
       'subtopics', 'thalia_title', 'thalia_subtitle', 'thalia_authors',
       'thalia_serie', 'thalia_successor', 'thalia_description',
       'thalia_languages', 'thalia_average_rating', 'thalia_total_rating',
       'thalia_publisher', 'thalia_sales_rank', 'thalia_coverlink',
       'thalia_also_read', 'thalia_bought_with', 'thalia_genres',
       'thalia_min_age', 'thalia_max_age'],
      dtype='object')

In [97]:
forQuery.to_csv("QueryForIsbn.csv", sep="|", index=False)

In [98]:
forQuery

,itemID,isbn,title,authors,publisher,main topic,subtopics,thalia_title,thalia_subtitle,thalia_authors,...,thalia_average_rating,thalia_total_rating,thalia_publisher,thalia_sales_rank,thalia_coverlink,thalia_also_read,thalia_bought_with,thalia_genres,thalia_min_age,thalia_max_age
1538,14576,9783641100148,Unsterblich 01 - Tor der Dämmerung,[Julie Kagawa],Heyne Taschenbuch,YFE,"[5AQ, FM, YFE, YFH]",Unsterblich - Tor der Dämmerung,Band 1 - Roman,[Julie Kagawa],...,4.5,43,Random House ebook,15835,https://assets.thalia.media/img/artikel/b178ab...,"[9783641171810, 9783736306103, 9783732500239, ...",9783641122713,"[Jugendbücher, Romane & Erzählungen, Fantasy &...",14,<NA>
1553,15581,9783608105353,Der Herr der Ringe,[J. R. R. Tolkien],Klett-Cotta Verlag,FMB,"[FYT, YFH]",Der Herr der Ringe,In der Übersetzung von Margaret Carroux,[J. R. R. Tolkien],...,4.7,62,Klett Cotta,6415,https://assets.thalia.media/img/artikel/373ed8...,"[9783608101393, 9783426451311, 9783641139582, ...",9783608101386,"[Fantasy & Science Fiction, Fantasy, Kinderbüc...",<NA>,<NA>
1576,4586,9783966982573,Meine Freunde,[Viktoria Schlünzen],NOVA MD,YZG,"[5AF, YZS]",Meine Freunde,Ein Freundschaftsbuch - Freundebuch für Kinder...,[Viktoria Schlünzen],...,5.0,1,Nova Md,22875,https://assets.thalia.media/img/artikel/13d2f4...,"[4014489123606, 9783314104534, 4142777520001, ...",9783946739951,"[Kinderbücher, Reise & Abenteuer, Ratgeber, Fa...",5,7
1586,5368,9781444942552,Winter Stories,[Enid Blyton],Hachette Children's Book,YFA,"[5AH, 5HPD, 4CD, YFH, YFJ, YFU]",Winter Stories,Contains 30 classic tales,[Enid Blyton],...,NaN,<NA>,Hachette Children's Group,<NA>,https://assets.thalia.media/img/artikel/aad0ee...,"[9780140309577, 9780679883388, 9780007274772, ...",9783125737686,<NA>,6,11
1620,73840,9780711252172,Astrid Lindgren,[María Isabel Sánchez Vegara],Insel Verlag GmbH,JBSF1,"[5AF, JBSF1, YBCS, YBL, YNB, YNG, YNL, YNM, YN...",Astrid Lindgren,<NA>,[Maria Isabel Sanchez Vegara],...,NaN,<NA>,Frances Lincoln,<NA>,https://assets.thalia.media/img/artikel/269656...,"[9780711245730, 9780711243170, 9780711257580, ...",9783836582506,<NA>,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66052,14187,9781450008341,Remnant,[Lesley Barklay],Xlibris,FM,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66053,17607,9781450008501,"Ysstrhm 4, Second Quest",[Douglas Browning],Xlibris,FM,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66054,70985,9781450012119,"Ysstrhm 5, Flight's End",[Douglas Browning],Xlibris,FM,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
66059,37678,9789738882690,Timeless Fairy Tales,"[Brothers Grimm, Marie-Catherine Baroness D'Au...",MEDIAMORPHOSIS,YFA,[YFJ],Timeless Fairy Tales,<NA>,"[Brothers Grimm, Marie-Catherine Baroness D'Au...",...,NaN,<NA>,Mediamorphosis,<NA>,https://assets.thalia.media/img/artikel/29a064...,"[9780141331201, 9780982499474, 9781680100280, ...",<NA>,<NA>,<NA>,<NA>


In [99]:
path = "../Dataset/API Datasets/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

df = pd.DataFrame()
for i in tqdm(file_list_py):
    data = pd.read_csv(path + i, sep="|", encoding='utf-16', error_bad_lines=False)
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)
apidata = df

C:\Users\rmaql\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\rmaql\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [100]:
apidata

,titleQueried,titleFound,author,language,description,thumbnail,maturityRating,publisher,publishedDate,pageCount,isEbook,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,un,A brand-new story in the best-selling Princess...,http://books.google.com/books/content?id=2cPHA...,NOT_MATURE,Corgi,2007,128.0,False,9.780553e+12
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,un,A brand-new story in the best-selling Princess...,http://books.google.com/books/content?id=WY6VX...,NOT_MATURE,Random House,2010-11-30,128.0,True,9.781409e+12
2,Princess Poppy: The Big Mix Up,Princess Poppy,Janey Louise Jones,un,A brand-new story in the best-selling Princess...,NaN,NOT_MATURE,Corgi Childrens,2010-08-02,128.0,False,9.780553e+12
3,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,un,NaN,NaN,NOT_MATURE,NaN,2007,118.0,False,9.780553e+12
4,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,http://books.google.com/books/content?id=xC7rN...,NOT_MATURE,Random House,2010-11-30,128.0,True,9.781409e+12
...,...,...,...,...,...,...,...,...,...,...,...,...
888730,Big Nate: Triple Decker Box Set: Big Nate: Wha...,Komplexität und Widerspruch in der Architektur,Robert Venturi,en,"Robert Venturi gehört zu den Architekten, die ...",http://books.google.com/books/content?id=dDS_A...,NOT_MATURE,Birkhäuser,2007-03-07,231.0,False,NaN
888731,Big Nate: Triple Decker Box Set: Big Nate: Wha...,Schwarz. Weiß. Tot.,Deon Meyer,de,Superintendent John October hat vor elf Jahren...,http://books.google.com/books/content?id=krbXA...,NOT_MATURE,Aufbau Digital,2010-04-23,282.0,True,9.783841e+12
888732,Big Nate: Triple Decker Box Set: Big Nate: Wha...,Das Kino des George Lucas,NaN,en,NaN,http://books.google.com/books/content?id=pkBuA...,NOT_MATURE,NaN,2005,263.0,False,9.783896e+12
888733,The Crossing: Premium Hardcover Edition,The Crossing,Jo Wilde,en,I had my whole world ahead of me. Eager to tak...,http://books.google.com/books/content?id=Ddtiz...,NOT_MATURE,NaN,2021-03-30,312.0,False,9.781035e+12


In [87]:
lookup_thalia = pd.read_csv("../Dataset/From Other Teams/header_items_lookup_df.csv", lineterminator='\n')
thalia = pd.read_csv("../Dataset/From Other Teams/header_items_df.csv", lineterminator='\n')

In [88]:
thalia2 = thalia.join(lookup_thalia.set_index('headerID')['itemID'], on='headerID')
thalia2.drop(['Unnamed: 0'], axis=1, inplace=True)
thalia2.head(3)

,headerID,title,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
0,0,and the word became a story,{'the author'},{'books on demand'},{'Englisch'},{304},{''},{Timestamp('2018-04-17 00:00:00')},"['FM', 'FN', 'FL']",at the beginning was the wordand from the word...,5649
1,1,evvai,{'cristina polacchini'},{'lulucom'},{'Italienisch'},{106},{''},{Timestamp('2015-08-01 00:00:00')},['YFB'],amicizia scuola amore alti e bassi in and out ...,77497
2,2,leviathan wakes calibans war abaddons gate no...,{'james s a corey'},{'orbit'},{'Englisch'},{1792},{''},{Timestamp('2019-11-19 00:00:00')},['FLS'],for the first tim the first three novels in co...,56963


In [92]:
print(leftout.shape)
leftout.head(3)

(4970, 7)


,itemID,title,author,publisher,main topic,subtopics,key
0,64544,Spiele-Klassiker Gummitwist-Set,Birgit Fuchs,moses. Verlag GmbH,YBL,,spieleklassikergummitwistsetbirgitfuchs
1,23197,Nichts,Janne Teller,dtv Verlagsgesellschaft,YFS,5AQ 1DND YFB YFS,nichtsjanneteller
2,2417,A Court of Thorns and Roses. Acotar Adult Edition,Sarah J. Maas,Bloomsbury UK,YFH,FMR,acourtofthornsandrosesacotaradulteditionsarahj...


In [93]:
mask = thalia2['itemID'].isin(leftout['itemID'])
booksInCommon2, left2 = thalia2[mask], thalia2[~mask]

In [94]:
booksInCommon2

,headerID,title,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
15,15,1 2 3 a calmer me helping children cope when e...,{'colleen a patterson brenda miles claire keay'},{'magination pr'},{'Englisch'},{32},{'4 - 8 Jahr(e)'},{Timestamp('2015-09-22 00:00:00')},"['YF', 'YXE']",NaN,63392
27,27,100 % mallorca ausmalbuch,{'lluisa calafat margalida castells'},{'dix'},"{'Deutsch, Englisch, Spanisch'}",{44},{'6 - 8 Jahr(e)'},{Timestamp('2017-05-01 00:00:00')},['Y'],ausmalbuch zum farbigen reisefuehrer hier kann...,75819
34,34,100 first words for toddlers english spanish...,{'jayme yannuzzi'},{'rockridge pr'},{'Englisch'},{56},{'0 - 3 Jahr(e)'},{Timestamp('2021-03-02 00:00:00')},"['YBLA', 'YPCA21']",NaN,52611
55,55,100% maedchen,{'caroline de hugo sacha maris noémie andré ca...,{'moses verlag gmbh'},{'Deutsch'},{224},{'8 - 10 Jahr(e)'},{Timestamp('2017-04-30 00:00:00')},"['YN', '5AL', 'YBL']",,32503
92,92,11,{'david williams'},{'wild wolf publishing'},{'Deutsch'},{192},{'14 - 99 Jahr(e)'},{Timestamp('2021-03-23 00:00:00')},['FXM'],,53219
...,...,...,...,...,...,...,...,...,...,...,...
71616,71616,¡feliz cumpleaños dragón happy birthday dragon...,{'steve herman'},{'lightning source inc'},{'Spanisch'},{46},{'4 - 8 Jahr(e)'},{Timestamp('2020-03-10 00:00:00')},"['YF', '5HKA']",tener un dragón de mascota es muy divertidopue...,68222
71639,71639,¡serendip al ataque / serendip on the attack,{'oscar julve jaume copons'},{'beascoa'},{'Spanisch'},{96},{'7 - 10 Jahr(e)'},{Timestamp('2020-07-21 00:00:00')},['YFC'],ulises y lía se apuntan a aventura total una e...,25319
71647,71647,àis journal,{'the kite'},{'authorhouse uk'},{'Englisch'},{412},{''},{Timestamp('2011-06-24 00:00:00')},['FM'],can cross galactic inhabitants interact with o...,23950
71661,71661,éveil,{'aurora clerc'},{'books on demand'},{''},{68},{''},{Timestamp('2021-03-15 00:00:00')},"['3KLF', 'FMH', 'FMB', '5AX', 'FT']",NaN,6755


In [112]:
booksInCommon2.head(3)

,headerID,title,author,publisher,language,number_pages,recommended_age,release_date,mt_st_,description_,itemID
15,15,1 2 3 a calmer me helping children cope when e...,{'colleen a patterson brenda miles claire keay'},{'magination pr'},{'Englisch'},{32},{'4 - 8 Jahr(e)'},{Timestamp('2015-09-22 00:00:00')},"['YF', 'YXE']",NaN,63392
27,27,100 % mallorca ausmalbuch,{'lluisa calafat margalida castells'},{'dix'},"{'Deutsch, Englisch, Spanisch'}",{44},{'6 - 8 Jahr(e)'},{Timestamp('2017-05-01 00:00:00')},['Y'],ausmalbuch zum farbigen reisefuehrer hier kann...,75819
34,34,100 first words for toddlers english spanish...,{'jayme yannuzzi'},{'rockridge pr'},{'Englisch'},{56},{'0 - 3 Jahr(e)'},{Timestamp('2021-03-02 00:00:00')},"['YBLA', 'YPCA21']",NaN,52611


In [126]:
booksInCommon3 = booksInCommon2.copy()
booksInCommon3['author'] = booksInCommon3['author'].str.replace("\{|\}|\'", "")
booksInCommon3['publisher'] = booksInCommon3['publisher'].str.replace("\{|\}|\'", "")
booksInCommon3['language'] = booksInCommon3['language'].str.replace("\{|\}|\'", "")
booksInCommon3 = booksInCommon3.drop(['number_pages', 'recommended_age', 'release_date', 'mt_st_',
       'description_', 'headerID'], axis = 1)
booksInCommon3 = booksInCommon3[['itemID', 'title', 'author', 'publisher', 'language']]
print(booksInCommon3.shape)
booksInCommon3.head(3)

(4898, 5)


,itemID,title,author,publisher,language
15,63392,1 2 3 a calmer me helping children cope when e...,colleen a patterson brenda miles claire keay,magination pr,Englisch
27,75819,100 % mallorca ausmalbuch,lluisa calafat margalida castells,dix,"Deutsch, Englisch, Spanisch"
34,52611,100 first words for toddlers english spanish...,jayme yannuzzi,rockridge pr,Englisch


In [127]:
leftout2 = leftout[~leftout['itemID'].isin(booksInCommon2['itemID'].tolist())]
leftout2.reset_index(inplace=True)
leftout2.drop(['index', 'main topic', 'subtopics', 'key'], axis=1, inplace=True)
leftout2

C:\Users\rmaql\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,itemID,title,author,publisher
0,76416,Warhammer 40.000 - Die Faust des Verräters,Sandy Mitchell,Black Library
1,17477,Advisor Because Freakin Badass Is Not an Offic...,Advisor Notebook,INDEPENDENTLY PUBLISHED
2,74108,Galens Suche,Erin Elder,epubli
3,4918,Moonlight and Aleena: A Tale of Two Friends,"L. P. Dover, Anna-Grace Dover",LIGHTNING SOURCE INC
4,18263,Libri da colorare per bambini di 2 anni (Fitne...,Gino Bianchi,COLORING PAGES
...,...,...,...,...
67,1641,Libro da colorare per bambini di 4-5 anni (Fra...,Gino Bianchi,COLORING PAGES
68,68279,Libro da colorare per bambini di 4-5 anni (Api...,Gino Bianchi,COLORING PAGES
69,34645,Libro da colorare per bambini di 4-5 anni (Gre...,Gino Bianchi,COLORING PAGES
70,360,The Seventeen Series Ultimate Short Story Coll...,A D Starrling,AD STARRLING


In [129]:
mergedMore = pd.concat([booksInCommon3, leftout2])
mergedMore

,itemID,title,author,publisher,language
15,63392,1 2 3 a calmer me helping children cope when e...,colleen a patterson brenda miles claire keay,magination pr,Englisch
27,75819,100 % mallorca ausmalbuch,lluisa calafat margalida castells,dix,"Deutsch, Englisch, Spanisch"
34,52611,100 first words for toddlers english spanish...,jayme yannuzzi,rockridge pr,Englisch
55,32503,100% maedchen,caroline de hugo sacha maris noémie andré camil,moses verlag gmbh,Deutsch
92,53219,11,david williams,wild wolf publishing,Deutsch
...,...,...,...,...,...
67,1641,Libro da colorare per bambini di 4-5 anni (Fra...,Gino Bianchi,COLORING PAGES,NaN
68,68279,Libro da colorare per bambini di 4-5 anni (Api...,Gino Bianchi,COLORING PAGES,NaN
69,34645,Libro da colorare per bambini di 4-5 anni (Gre...,Gino Bianchi,COLORING PAGES,NaN
70,360,The Seventeen Series Ultimate Short Story Coll...,A D Starrling,AD STARRLING,NaN


In [130]:
mergedMore.isnull().sum()

itemID        0
title         0
author        0
publisher     0
language     72
dtype: int64

In [131]:
mergedMore.to_csv("QueryForTitleAuthor.csv", sep="|", index=False)
# forQuery.to_csv("QueryForIsbn.csv", sep="|", index=False)

In [132]:
mergedMore.to_excel("QueryForTitleAuthor.xlsx")

In [133]:
a = pd.read_excel("./QueryForTitleAuthor.xlsx")

In [135]:
a.drop(['Unnamed: 0'], axis=1, inplace=True)

In [137]:
a.to_csv("QueryForTitleAuthor.csv", sep="|", index=False)

,itemID,title,author,publisher,language
0,63392,1 2 3 a calmer me helping children cope when e...,colleen a patterson brenda miles claire keay,magination pr,Englisch
1,75819,100 % mallorca ausmalbuch,lluisa calafat margalida castells,dix,"Deutsch, Englisch, Spanisch"
2,52611,100 first words for toddlers english spanish...,jayme yannuzzi,rockridge pr,Englisch
3,32503,100% maedchen,caroline de hugo sacha maris noémie andré camil,moses verlag gmbh,Deutsch
4,53219,11,david williams,wild wolf publishing,Deutsch
...,...,...,...,...,...
4965,1641,Libro da colorare per bambini di 4-5 anni (Fra...,Gino Bianchi,COLORING PAGES,NaN
4966,68279,Libro da colorare per bambini di 4-5 anni (Api...,Gino Bianchi,COLORING PAGES,NaN
4967,34645,Libro da colorare per bambini di 4-5 anni (Gre...,Gino Bianchi,COLORING PAGES,NaN
4968,360,The Seventeen Series Ultimate Short Story Coll...,A D Starrling,AD STARRLING,NaN
